## Creating the AWS Serverless Application Model (SAM) application

### [Verify pre-requisites for AWS SAM](https://docs.aws.amazon.com/serverless-application-model/latest/developerguide/serverless-sam-cli-install.html#serverless-sam-cli-install-prerequisites)
SAM requires python 2.7 or 3.6 and pip to be installed to continue.

In [ ]:
import boto3
import os
import sagemaker
from sagemaker import get_execution_role
import project_path # path to helper methods
from lib import workshop

role = get_execution_role()

s3 = boto3.resource('s3')
cfn = boto3.client('cloudformation')

# use the region-specific sample data bucket
region = boto3.Session().region_name
stack_name = 'serverless-hello'

## [Create S3 Bucket](https://docs.aws.amazon.com/AmazonS3/latest/gsg/CreatingABucket.html)

We will create an S3 bucket that will be used throughout the workshop for storing our data.

[s3.create_bucket](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/s3.html#S3.Client.create_bucket) boto3 documentation

In [ ]:
bucket = workshop.create_bucket_name('sam-')
s3.create_bucket(Bucket=bucket, CreateBucketConfiguration={'LocationConstraint': region})
print(bucket)

## Verify Python and pip versions

Both Python and pip will be installed when using SageMaker managed notebooks.

In [ ]:
!python --version
!pip --version

### [Installing AWS SAM](https://docs.aws.amazon.com/serverless-application-model/latest/developerguide/serverless-sam-cli-install.html)

The primary distribution method for the AWS SAM CLI on Linux, Windows, and macOS is pip, a package manager for Python that provides an easy way to install, upgrade, and remove Python packages and their dependencies.

Pre-requistites:
* Docker
* [AWS Command Line Interface (AWS CLI)](https://docs.aws.amazon.com/cli/latest/userguide/)
* (Pip only) Python 2.7 or Python 3.6

In [ ]:
!pip install aws-sam-cli

### Verify the latest version of SAM is installed 

In [ ]:
%%bash
export PATH=$PATH:/home/ec2-user/.local/bin
sam --version
sam init -r python3.6 -n serverless-todo

In [ ]:
!cat serverless-todo/README.md

## Load required dependencies locally to the application

In [ ]:
!pip install -r serverless-todo/hello_world/requirements.txt -t serverless-todo/hello_world/

In [ ]:
!pygmentize serverless-todo/hello_world/app.py

### [Validate template](https://docs.aws.amazon.com/AWSCloudFormation/latest/UserGuide/using-cfn-validate-template.html)

The `aws cloudformation validate-template` command is designed to check only the syntax of your template. It does not ensure that the property values that you have specified for a resource are valid for that resource. Nor does it determine the number of resources that will exist when the stack is created.

In [ ]:
!aws cloudformation validate-template --template-body file://serverless-todo/template.yaml

### [Package deployment](https://docs.aws.amazon.com/AWSCloudFormation/latest/UserGuide/using-cfn-cli-package.html)
<a id='package'></a>

For some resource properties that require an Amazon S3 location (a bucket name and filename), you can specify local references instead. For example, you might specify the S3 location of your AWS Lambda function's source code or an Amazon API Gateway REST API's OpenAPI (formerly Swagger) file. Instead of manually uploading the files to an S3 bucket and then adding the location to your template, you can specify local references, called local artifacts, in your template and then use the package command to quickly upload them. A local artifact is a path to a file or folder that the package command uploads to Amazon S3. For example, an artifact can be a local path to your AWS Lambda function's source code or an Amazon API Gateway REST API's OpenAPI file.

In [ ]:
!aws cloudformation package \
    --template-file serverless-todo/template.yaml \
    --output-template-file serverless-todo/sam-template.yaml \
    --s3-bucket $bucket \
    --s3-prefix serverless

### [Deploy Application](https://docs.aws.amazon.com/AWSCloudFormation/latest/UserGuide/using-cfn-cli-deploy.html)

AWS CloudFormation requires you to use a change set to create a template that includes transforms. Instead of independently creating and then executing a change set, use the `aws cloudformation deploy` command. When you run this command, it creates a change set, executes the change set, and then terminates. This command reduces the numbers of required steps when you create or update a stack that includes transforms.

In [ ]:
!aws cloudformation deploy \
    --template-file serverless-todo/sam-template.yaml \
    --stack-name serverless-hello \
    --capabilities CAPABILITY_IAM

### [View Output](https://docs.amazonaws.cn/en_us/AWSCloudFormation/latest/UserGuide/using-cfn-describing-stacks.html)

The `aws cloudformation describe-stacks` command provides information on your running stacks. You can use an option to filter results on a stack name. This command returns information about the stack, including the name, stack identifier, and status.

In [ ]:
!aws cloudformation describe-stacks --stack-name $stack_name --region $region

### Get the API Endpoint created from the SAM deployment

In [ ]:
stacks = cfn.describe_stacks(StackName=stack_name)
stack = stacks["Stacks"][0]

for output in stack["Outputs"]:
    if output["OutputKey"] == 'HelloWorldApi':
        api_endpoint = output["OutputValue"]
#     print('%s=%s (%s)' % (output["OutputKey"], output["OutputValue"], output["Description"]))
    
print(api_endpoint)

### Call REST API

After we create and deploy the project we will install [httpie](https://httpie.org/) which is a framework that consists of a single `http` command designed for interaction with HTTP servers, RESTful APIs, and web services. Replace the variable `{{api-gw-address}}` with the `HelloWorldApi` in `Outputs` above.

In [ ]:
!pip install httpie

In [ ]:
print('calling {0}'.format(api_endpoint))
!http $api_endpoint

### Create a TODO application from the Hello World example

Now that you have learned how to create a serverless application with the Hello World example it's time for you to create a todo application from the example. There is sample code for a [todo](./todos) application.

Steps to create
* Modify the `template.yaml` to include the REST paths for GET/LIST/PUT/DELETE for the application and include the DynamoDB table in the template that will be used to store and retrieve the todos.
* Create a Cognito user pool to provide authentication capabilties to your API.
* Modify the `template.yaml` file to include a `CustomAuthorizer` for the PUT endpoint.
* Run through example REST calls using `httpie`
* Cleanup

When you are done modifying the `template.yaml` we can go ahead an repackage and deploy the template to update the service. [CloudFormation Package](#package)

### Create a todo

In [ ]:
echo '{ "text": "My First Todo" }' | http POST https://{api-url}/dev/todos
http https://{api-url}/dev/todos

### List the todos available

In [ ]:
!http POST https://{api-url}/dev/todos

Now you can see how easy it is to create a microservice using Lambda but what if we wanted to only allow authenticated users to have the ability to create todos? We can create a Cognito user pool and use it as an authorizer to the API. Create a new Cognito user pool and hook it up to the create method of the service. 

Change the template.yml file to add the authorizer like below.

```
    authorizer:
        type: COGNITO_USER_POOLS
        authorizerId:
            Ref: TodoApiGatewayAuthorizer
```

Let's redeploy the service and try and add a todo

We can now package and deploy the updated service. [CloudFormation Package](#package)

In [ ]:
!echo '{ "text": "My First Todo" }' | http POST https://{api-url}/dev/todos

This should have returned an HTTP/1.1 401 Unauthorized because you did not send the appropriate Authorization header on the call. We will simulate a user logging into the application and passing the bearer token in the headers of the request. Running the 3 methods below will simulate creating a user, confirming the user, and generating the appropriate auth for the user to be able to create the todo. Grab the IdToken generated and add it to the headers like the command below.

### Create user in Cognito

In [ ]:
username = '{{username}}'
password = '{{password}}'

In [ ]:
!aws cognito-idp sign-up --region $region --client-id {cognito-client-id} --username $username --password $password
                                   

In [ ]:
%%writefile auth.json

{
    "UserPoolId": "{{your-cognito-user-pool}}",
    "ClientId": "{{your-cognito-client-id}}",
    "AuthFlow": "ADMIN_NO_SRP_AUTH",
    "AuthParameters": {
        "USERNAME": "{{username}}",
        "PASSWORD": "{{passsword}"
    }
}


### Confirm the sign up of the user

In [ ]:
!aws cognito-idp admin-confirm-sign-up --region $region --user-pool-id {cognito-user-pool} --username {{username}}


### Initiate the authentication to generate the token

In [ ]:
!aws cognito-idp admin-initiate-auth --region $region --cli-input-json file://auth.json

In [ ]:
!echo '{ "text": "My Authenticated Todo" }' | http POST https://{api-url}/dev/todos Authorization:"Bearer {your-idtoken}"

Now you should have created a new todo after the authorizer approves the token.

### Register a user

> aws cognito-idp sign-up --region {your-aws-region} --client-id {your-client-id} --username admin@example.com --password {password}

### Confirm user registration

> aws cognito-idp admin-confirm-sign-up --region {your-aws-region} --user-pool-id {your-user-pool-id} --username admin@example.com

### Authenticate (get tokens)

> aws cognito-idp admin-initiate-auth --region {your-aws-region} --cli-input-json file://auth.json

### Where auth.json is:

>{
    "UserPoolId": "{your-user-pool-id}",
    "ClientId": "{your-client-id}",
    "AuthFlow": "ADMIN_NO_SRP_AUTH",
    "AuthParameters": {
        "USERNAME": "admin@example.com",
        "PASSWORD": "{password}"
    }
}

### You should get a response like this if everything is set up correctly:

>{
    "AuthenticationResult": {
        "ExpiresIn": 3600,
        "IdToken": "{your-idtoken}",
        "RefreshToken": "{your-refresh-token}",
        "TokenType": "Bearer",
        "AccessToken": "{your-access-token}"
    },
    "ChallengeParameters": {}
}

## Cleanup

In [ ]:
!aws cloudformation delete-stack --stack-name $stack_name

In [ ]:
workshop.delete_bucket_completely(bucket)